# Auto time series forecaster example

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Automatic time series forecasting 
import AutoTimeSeriesForecaster
plt.style.use('ggplot')
# Temp. warning filter
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Creating a dataset of ED admissions (not using real data)
df_in = pd.DataFrame()
df_in.index = pd.date_range(start = '2021-01-01', end = '2023-04-10', freq='d')
# ED Admissions, random number between 30 and 50
df_in['ED Admissions'] = np.random.randint(low=30, high=50, size=len(df_in))
# ED Admissions 50% higher on weekends
df_in.loc[df_in.index.weekday.isin([5, 6]), 'ED Admissions'] *= 1.5
# ED Admissions 100% higher on New Years day
df_in.loc[df_in.index.dayofyear == 1, 'ED Admissions'] *= 2
# Ensure values are integers
df_in = df_in.astype('int')
# Display first 5 rows
df_in.head()

,ED Admissions
2021-01-01,98
2021-01-02,45
2021-01-03,49
2021-01-04,46
2021-01-05,33


In [3]:
# Instantiate instance of the AutoTS class.
AutoTS = AutoTimeSeriesForecaster.AutoTS(input_df = df_in, # Input dataframe
                           target = 'ED Admissions', # Target variable name
                           freq = 'd', # Frequency of data ('d' - Daily)
                           sp = 7, # sp - Seasonal Period
                           holidays_list = ["New Year's Day"] ) # Include effect of New Years day

# Carry out cross validation on training data
AutoTS.ts_cv(forecast_horizon = 14, # forecast 2 weeks each fold
             folds = 5, # Number of cross validation folds
             perf_metric='MAPE') # Performance metric to minimise when selecting chosen model

2023-04-10 18:38:19 prophet INFO: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2023-04-10 18:38:20 prophet INFO: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2023-04-10 18:38:21 prophet INFO: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2023-04-10 18:38:21 prophet INFO: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2023-04-10 18:38:22 prophet INFO: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2023-04-10 18:38:23 prophet INFO: Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [4]:
# Print cross validation results (showing MAPE)
AutoTS.cv_summary

,MAPE,MSE,MAE
model,,,
Facebook Prophet,0.142329,49.636957,6.015443
ETS,0.147714,52.638129,6.199171
ARIMA,0.150571,52.425114,6.280657
STL,0.178729,112.496614,8.189314
Seasonal Naive,0.183243,117.528571,8.300000


In [5]:
# Produce forecasts (using default model i.e., lowest MAPE during cross validation.)
AutoTS.forecast(timesteps = 365, # forecast horizon
                pi = 0.95) # exog_to_forecast='Responses') # Prediction interval width

In [6]:
AutoTS.forecast_df.head()

,yhat,yhat_lower,yhat_upper
2023-04-11,38.485367,25.323859,51.336556
2023-04-12,38.167025,25.311219,51.046296
2023-04-13,37.231889,25.115846,49.793774
2023-04-14,38.338302,25.544772,51.218870
2023-04-15,56.956426,43.959203,69.518214


In [7]:
# Forecast plot
_, ax = plt.subplots(figsize = (20, 5))

prediction = AutoTS.forecast_df
ax.fill_between(x = prediction.index, y1 = prediction.yhat_lower, 
               y2 = prediction.yhat_upper, alpha = 0.3, label = '95% Prediction interval')
ax.plot(prediction.index, prediction.yhat, label = 'Forecast')
df_in[['ED Admissions']].plot(figsize = (15, 5), ax = ax)
ax.grid(alpha = 0.75)
ax.legend(loc='upper left')
ax.set_ylabel('Daily ED admissions')
ax.set_title('Artificial daily ED admissions data example - One year forecast')
plt.show()